# Meander Maker

If you haven't ever used the `googlemaps` python package, there are a few small setup steps to take first. You'll need to install it by running the next cell. Additionally, we will use `gmplot` to render the outputs. Finally, generate an API key to interface with Google's services. (There is a free tier, but if you use it a lot, they will start charging you for access.)

In [35]:
!pip install googlemaps
!pip install gmplot

    100% |████████████████████████████████| 122kB 8.3MB/s 
  Stored in directory: /Users/dakaspar/Library/Caches/pip/wheels/81/6a/76/4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot


In [36]:
import pandas as pd
import googlemaps
from datetime import datetime
from gmplot import gmplot

df = pd.read_csv('warehouse_district.csv', index_col='place')

If you would like to tinker around with the code, you'll need your own API key for Google Maps. You can register one from this address: [https://developers.google.com/places/web-service/get-api-key](https://developers.google.com/places/web-service/get-api-key). Once you have one, create a file in the directory containing this notebook called `.secret.key` and simply paste in your newly generated API key, no quotes or line breaks or anything. Save, exit, and you should be good to go.

In [3]:
with open('.secret.key') as f:
    api_key = f.read().strip()

gmaps = googlemaps.Client(key=api_key)

Let's decide which topic to explore:

In [28]:
keyword = input('What theme walk would you like to explore today?')

What theme walk would you like to explore today? bookstore


In [29]:
temp = gmaps.places_nearby((47.6225507, -122.5167143), radius=3000, keyword=keyword)

In [30]:
[x['name'] for x in temp['results']]

['Eagle Harbor Book Co', 'Backstreet Beat', 'Christian Science Reading Room']

In [ ]:
def nearify(start=None, keyword=keyword):
    """
    Used to avoid multiple locations with the same name. This will auto-search 
    for the closest matches to the starting location, even when other (potentially more 
    famous) locations exist further away.
    """
    if start is None:
        start = dest_list[0]
    
    gmaps.places_nearby()
    

In [8]:
dest_list = ['Genoa Cellars', 'Gorman -Old Scratch Winery', 'Long Cellars', 'LOVE THAT RED', 'Ancestry Cellars']

def round_trip(circle=True, dest_list=dest_list):
    """Append the Start location to the end of the list, resulting in a loop."""
    return dest_list.extend(dest_list[0])

def walk(dest_list, verbose=False):
    """Given a list of places to visit, return the walking dist & time."""

    if len(dest_list) > 10:
        return "There is a maximum of 10 stops per adventure."
    
    #assert len(dest_list) < 11, "There is a maximum of 10 stops per adventure."
    
    start, stop = dest_list[0], dest_list[-1]
    
    waypoints = None
    if len(dest_list) > 2:
        waypoints = dest_list[1:-1]
    
    directions_result = gmaps.directions(
        start, stop, mode="walking", waypoints=waypoints, optimize_waypoints=True
    )
    
    if verbose:
        dist = 0
        time = 0
        for leg in directions_result[0]['legs']:
            td = leg['distance']['value']
            tt = leg['duration']['value']
            print(f'+{td} m --and-- +{round(tt/60, 2)} min')
            dist += td
            time += tt
        print(f'total dist: {dist} m \nest time: {round(time / 60, 1)} min')
    return directions_result

In [31]:
# path = walk(dest_list)
# path

In [33]:
path[0]['bounds']

{'northeast': {'lat': 47.8851198, 'lng': -119.9709734},
 'southwest': {'lat': 47.17528739999999, 'lng': -122.1541535}}

In [65]:
gmaps.find_place('LOVE THAT RED', input_type='textquery', location_bias='point:47.769302, -122.148902')

{'candidates': [{'place_id': 'ChIJ1c9qHpgOkFQRPaH6VxO7Lhs'}], 'status': 'OK'}

In [66]:
gmaps.place('ChIJ1c9qHpgOkFQRPaH6VxO7Lhs')

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'D100',
    'short_name': 'D100',
    'types': ['subpremise']},
   {'long_name': '19501', 'short_name': '19501', 'types': ['street_number']},
   {'long_name': '144th Avenue Northeast',
    'short_name': '144th Ave NE',
    'types': ['route']},
   {'long_name': 'North Industrial',
    'short_name': 'North Industrial',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Woodinville',
    'short_name': 'Woodinville',
    'types': ['locality', 'political']},
   {'long_name': 'King County',
    'short_name': 'King County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Washington',
    'short_name': 'WA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '98072', 'short_name': '98072', 'types': ['postal_code']}],
  'adr_address': 'Back Entrance, <span c

In [79]:
gmaps.directions('LOVE THAT RED', 'Genoa Cellars', mode='walking', waypoints=[])

[{'bounds': {'northeast': {'lat': 47.7697239, 'lng': -122.1504484},
   'southwest': {'lat': 47.7690488, 'lng': -122.1514725}},
  'copyrights': 'Map data ©2019',
  'legs': [{'distance': {'text': '0.1 mi', 'value': 201},
    'duration': {'text': '2 mins', 'value': 147},
    'end_address': '19501 144th Ave NE C1100, Woodinville, WA 98072, USA',
    'end_location': {'lat': 47.7690489, 'lng': -122.1504484},
    'start_address': 'Back Entrance, 19501 144th Ave NE D100, Woodinville, WA 98072, United States',
    'start_location': {'lat': 47.7697232, 'lng': -122.1507996},
    'steps': [{'distance': {'text': '164 ft', 'value': 50},
      'duration': {'text': '1 min', 'value': 34},
      'end_location': {'lat': 47.7697239, 'lng': -122.1514725},
      'html_instructions': 'Head <b>west</b>',
      'polyline': {'points': 'w_qbHnqphV?dC'},
      'start_location': {'lat': 47.7697232, 'lng': -122.1507996},
      'travel_mode': 'WALKING'},
     {'distance': {'text': '246 ft', 'value': 75},
      'dura

In [61]:
gmap = gmplot.GoogleMapPlotter(47.7690488, -122.1514703, zoom=15, apikey=api_key)

In [59]:
gmap.plot([47.7690488], [-122.1514703], 'cornflowerblue', edge_width=10)
gmap.scatter([47.7690488], [-122.1514703], '#3B0B39', size=40, marker=False)
gmap.scatter([47.7690488], [-122.1514703], 'k', marker=True)
#gmap.heatmap(heat_lats, heat_lngs)

In [60]:
gmap.draw("mymap.html")

In [66]:
path = gmap.

In [67]:
path

[]